#### **`Dependencies`**

In [1]:
%%bash
# delete all '__pycache__' directories throughout the project
cd ..
rm -rf `find . -type d -name __pycache__`

# output the file structure from the project's root directory
tree

.
├── LICENSE
├── Makefile
├── README.md
├── artifacts
│   └── model.pkl
├── config.yaml
├── data
│   └── raw.parquet
├── notebooks
│   └── rental-prediction-service.ipynb
├── poetry.lock
├── pyproject.toml
├── src
│   ├── __init__.py
│   ├── app.py
│   ├── config.py
│   ├── data.py
│   ├── database.py
│   ├── logger.py
│   ├── model.py
│   ├── model_builder.py
│   ├── model_inference.py
│   ├── run_model_builder.py
│   └── run_model_inference.py
└── tests
    └── __init__.py

6 directories, 21 files


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import warnings

import numpy as np
import pandas as pd

from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

# rental-prediction-service modules
from src.data import encode_neighborhood_ids, preprocess_data
from src.database import read_table
from src.model import compute_rsquared, split_data
from src.model_inference import ModelInferenceService

warnings.filterwarnings("ignore")

In [4]:
# set the pd.DataFrame and pd.Series display options
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

#### **`Data ingestion and pre-processing`**

In [5]:
# raw data
read_table().head(3)

2025-01-18 15:20:24.572 | INFO     | src.database:read_table:133 - Fetching raw data from the 'postgres' database's 'rentals.raw' table.


,address,zip,neighborhood,neighborhood_id,year_built,area,rooms,bedrooms,bathrooms,balcony,storage,parking,furnished,garage,garden,energy,facilities,rent
0,1071 HN Amsterdam (Cornelis Schuytbuurt),1071 HN,Cornelis Schuytbuurt,48,1870,167.0,3,2,2.0,yes,no,no,yes,no,Not present,D,Roof terrace,4500
1,1071 HK Amsterdam (Concertgebouwbuurt),1071 HK,Concertgebouwbuurt,47,1890,150.0,3,2,2.0,yes,no,yes,yes,no,Not present,A,"Cable TV, Internet connection, Fireplace, Bath, Toilet",3450
2,1071 HK Amsterdam (Concertgebouwbuurt),1071 HK,Concertgebouwbuurt,47,1890,150.0,3,2,2.0,yes,no,yes,yes,no,Not present,A,"Cable TV, Internet connection, Fireplace, Bath, Toilet",3450


In [6]:
# pre-processed data
read_table().pipe(preprocess_data).head(3)

2025-01-18 15:20:29.750 | INFO     | src.database:read_table:133 - Fetching raw data from the 'postgres' database's 'rentals.raw' table.
2025-01-18 15:20:29.770 | INFO     | src.data:preprocess_data:80 - Validating, pre-processing, and transforming the raw data into ML-ready features and targets.


,year_built,area,bedrooms,bathrooms,furnished,storage,garage,parking,balcony,garden_size,neighborhood_id,rent
0,1870,167.0,2,2.0,True,False,False,False,True,0,48,4500
1,1890,150.0,2,2.0,True,False,False,True,True,0,47,3450
2,1923,90.0,2,1.0,True,False,False,False,True,0,106,2000


In [7]:
# machine learning-ready data
# NOTE: the 'neighborhood_id' feature has been encoded
read_table().pipe(preprocess_data).pipe(encode_neighborhood_ids).head(3)

2025-01-18 15:20:37.120 | INFO     | src.database:read_table:133 - Fetching raw data from the 'postgres' database's 'rentals.raw' table.
2025-01-18 15:20:37.143 | INFO     | src.data:preprocess_data:80 - Validating, pre-processing, and transforming the raw data into ML-ready features and targets.


,year_built,area,bedrooms,bathrooms,furnished,storage,garage,parking,balcony,garden_size,neighborhood_mean_area,neighborhood_mean_bedrooms,neighborhood_mean_bathrooms,neighborhood_mean_garden_size,rent
0,1870,167.0,2,2.0,True,False,False,False,True,0,127.50,2.500000,1.400000,5.766667,4500
1,1890,150.0,2,2.0,True,False,False,True,True,0,107.25,2.166667,1.166667,0.000000,3450
2,1923,90.0,2,1.0,True,False,False,False,True,0,122.00,2.111111,1.222222,10.444444,2000


#### **`Data splitting`**

In [8]:
# split the machine learning-ready data into train, validation, and test sets
Xtrain, Xval, Xtest, ytrain, yval, ytest = (
    read_table()
    .pipe(preprocess_data)
    .pipe(encode_neighborhood_ids)
    .pipe(split_data)
)

2025-01-18 15:21:01.712 | INFO     | src.database:read_table:133 - Fetching raw data from the 'postgres' database's 'rentals.raw' table.
2025-01-18 15:21:01.739 | INFO     | src.data:preprocess_data:80 - Validating, pre-processing, and transforming the raw data into ML-ready features and targets.
2025-01-18 15:21:01.775 | INFO     | src.model:split_data:38 - Splitting the ML-ready features and targets into train, validation, and test sets.


#### **`Model creation, evaluation, and hyperparameter tuning`**

In [ ]:
# fit a base model to the train set
model: XGBRegressor = XGBRegressor(base_score=0.5, n_jobs=-1)
model.fit(Xtrain, ytrain)

**`Model evaluation`**

In [ ]:
# compute the base model's test set R²
compute_rsquared(ytest, model.predict(Xtest))

**`Hyperparameter tuning`**

In [ ]:
# define the parameter search space
# NOTE: hyperparameters are optimized on a parameter-by-parameter basis, that is, ...
# 'n_estimators' is optimized 1st, then 'max_depth', then 'learning_rate' etc. 
search_space: dict[str, list[float | int]] = {
    # "n_estimators": [100, 200, 500], # R²: 
    # "max_depth": [3, 6, 10], # R²: 
    # "learning_rate": [0.05, 0.1, 0.2, 0.3], # R²: 
    # "gamma": [0.01, 0.1, 1], # R²: 
    # "min_child_weight": [0, 5, 20, 50] # R²: 
}

# instantiate an object of type, 'GridSearchCV'
gscv: GridSearchCV = GridSearchCV(
    estimator=XGBRegressor(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.3,
        gamma=0.1,
        base_score=0.5,
        n_jobs=-1
        ),
    param_grid=search_space,
    scoring="r2",
    refit="r2",
    cv=5,
    verbose=False
)

# fit the 'gscv' object to the train set
gscv.fit(Xtrain, ytrain)

# output the validation set R² and 'best' parameters
display(
    compute_rsquared(yval, gscv.best_estimator_.predict(Xval)),
    gscv.best_params_
)

In [ ]:
# extract the 'gscv' object's 'best' model
best_model: XGBRegressor = gscv.best_estimator_

# fit the 'best' model to the train set and evaluate it on the validation set
best_model.early_stopping_rounds = 20
best_model.fit(Xtrain, ytrain, eval_set=[(Xval, yval)], verbose=False)

# output the 'best' model's test set R²
# NOTE: hyperparameter tuning didn't make a difference, that is, ...
# the 'best' model's test set R², 0.82, is identical to the base model's test set R²
compute_rsquared(ytest, best_model.predict(Xtest))

#### **`Inference`**

In [9]:
# input record
record: dict[str, float | int | str] = {
    "year_built": 2016,
    "area": 105.0,
    "bedrooms": 3,
    "bathrooms": 2.0,
    "furnished": "no",
    "storage": "no",
    "garage": "yes",
    "parking": "yes",
    "balcony": "yes",
    "garden_size": 10.0,
    "neighborhood_id": 10
}

# instantiate an object of type, 'ModelInferenceService'
service: ModelInferenceService = ModelInferenceService()

# load the trained model 
service.load_model()

# output the prediction
service.predict(record)

2025-01-18 15:21:10.756 | INFO     | src.model_inference:load_model:42 - Checking if '~/artifacts/model.pkl' exists.
2025-01-18 15:21:10.757 | INFO     | src.model_inference:load_model:48 - '~/artifacts/model.pkl' found. Loading the trained ML model.
2025-01-18 15:21:10.766 | INFO     | src.model_inference:predict:64 - Generating the prediction...


3048

In [10]:
# compute the test set R² for 100 iterations
# NOTE: each iteration's test set will be randomly selected, so the chances of getting the same ...
# test set twice is slim to none
metrics: list[float] = []
for _ in range(100):
    _, _, Xtest, _, _, ytest = (
        read_table()
        .pipe(preprocess_data)
        .pipe(encode_neighborhood_ids)
        .pipe(split_data)
    )
    metric: float = compute_rsquared(ytest.values, service.model.predict(Xtest))
    metrics.append(metric)

# output the average test set R²
round(np.mean(metrics), 2)

2025-01-18 15:21:24.994 | INFO     | src.database:read_table:133 - Fetching raw data from the 'postgres' database's 'rentals.raw' table.
2025-01-18 15:21:25.020 | INFO     | src.data:preprocess_data:80 - Validating, pre-processing, and transforming the raw data into ML-ready features and targets.
2025-01-18 15:21:25.062 | INFO     | src.model:split_data:38 - Splitting the ML-ready features and targets into train, validation, and test sets.
2025-01-18 15:21:25.068 | INFO     | src.database:read_table:133 - Fetching raw data from the 'postgres' database's 'rentals.raw' table.
2025-01-18 15:21:25.089 | INFO     | src.data:preprocess_data:80 - Validating, pre-processing, and transforming the raw data into ML-ready features and targets.
2025-01-18 15:21:25.126 | INFO     | src.model:split_data:38 - Splitting the ML-ready features and targets into train, validation, and test sets.
2025-01-18 15:21:25.131 | INFO     | src.database:read_table:133 - Fetching raw data from the 'postgres' databas

0.83